In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.9 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from matplotlib.colors import ListedColormap
from sklearn.manifold import TSNE

In [6]:
np_load=np.load('result.npy')

In [7]:
np_load.shape

(64, 512, 1024)

In [8]:
result=np_load.mean(axis=1)

In [9]:
result.shape

(64, 1024)

In [62]:
final_embeddings=np.save('final_embeddings.npy',result)

In [10]:
df=pd.DataFrame(result)

In [12]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.153901,0.215569,0.091525,-0.227793,0.234947,0.005588,-0.121599,0.082298,-0.192559,-0.063096,...,0.106784,-0.039421,-0.207978,-0.097944,-0.216062,0.001153,0.035346,-0.007842,0.077081,0.121129
1,-0.183757,0.231597,0.176775,-0.271214,0.185417,-0.039271,-0.119767,0.052893,-0.178024,-0.012841,...,0.086609,-0.050969,-0.110115,-0.035936,-0.196804,0.009869,0.068853,-0.033861,0.136247,0.138711
2,-0.196014,0.248278,0.207246,-0.328365,0.211659,-0.015317,-0.094369,0.073252,-0.149931,-0.002763,...,0.029769,-0.043258,-0.100250,0.007566,-0.189010,-0.004896,0.035212,-0.045467,0.131107,0.187292
3,-0.191772,0.218992,0.155774,-0.274358,0.195491,-0.007639,-0.116360,0.038897,-0.164156,0.009429,...,0.059498,-0.003239,-0.159647,-0.037740,-0.191508,0.021841,0.060696,-0.037066,0.124531,0.157191
4,-0.215841,0.149433,0.113639,-0.323872,0.219298,-0.052719,-0.235411,0.010137,-0.138184,0.128476,...,0.016611,0.006175,-0.246824,0.020935,-0.153527,0.005034,0.018748,-0.015877,0.175337,0.194603


In [13]:
result

array([[-0.15390101,  0.2155694 ,  0.0915251 , ..., -0.0078424 ,
         0.07708057,  0.12112921],
       [-0.1837566 ,  0.2315973 ,  0.17677526, ..., -0.03386087,
         0.13624689,  0.1387108 ],
       [-0.1960141 ,  0.24827841,  0.20724647, ..., -0.04546675,
         0.13110738,  0.18729198],
       ...,
       [-0.15082671,  0.2209849 ,  0.13198237, ..., -0.0022533 ,
         0.13829784,  0.18966816],
       [-0.14465238,  0.22876933,  0.10029759, ..., -0.00781085,
         0.0750124 ,  0.1552795 ],
       [-0.23788695,  0.14499803,  0.10045211, ..., -0.02998046,
         0.09007409,  0.12487731]], dtype=float32)

In [14]:
label=['negative','positive','positive','positive','negative','positive','negative','negative','negative','negative','negative','positive','negative','positive','positive','positive','negative','negative','positive','negative','positive','positive','positive','positive','negative','negative','negative','positive','positive','negative','negative','positive','positive','positive','positive','negative','positive','positive','positive','positive','negative','positive','positive','negative','negative','positive','negative','positive','positive','negative','positive','positive','negative','negative','negative','negative','negative','positive','positive','negative','positive','negative','negative','positive']

In [17]:
class_label=[]
for i in label:
  if (i=='positive'):
    class_label.append(1)
  else:
    class_label.append(0)
print(class_label)

[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1]


In [20]:
X=result
X.shape

(64, 1024)

In [19]:
Y=np.array(class_label)

In [21]:
Y.shape

(64,)

In [63]:
from sklearn.model_selection import cross_val_score

models = {
    'Logistic Regression': LogisticRegression(C=0.01, max_iter=1000),
}

# 7. Evaluate with 10-fold Cross Validation
print("\nModel Comparison after Outlier Removal:\n")
for name, model in models.items():
    scores = cross_val_score(model, X, Y, cv=10, scoring='accuracy')
    print(f"{name:25s}: Mean Accuracy = {scores.mean():.4f} | Std = {scores.std():.4f}")



Model Comparison after Outlier Removal:

Logistic Regression      : Mean Accuracy = 0.5286 | Std = 0.0350


In [73]:
models = {
    'Logistic Regression': LogisticRegression(C=0.01,max_iter=1000),
    'SVM (Linear)': SVC(kernel='linear',C=10,gamma='scale'),
    'SVM (RBF)': SVC(kernel='rbf'),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    #'Naive Bayes': MultinomialNB(), # Removed as it's not suitable for data with negative values after PCA
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [77]:
print("Model Comparison using 10-Fold Cross Validation:\n")
for name, model in models.items():
    scores = cross_val_score(model, X, Y, cv=10, scoring='accuracy')
    print(f"{name:25s}: Mean Accuracy = {scores.mean():.4f} | Std = {scores.std():.4f}")

Model Comparison using 10-Fold Cross Validation:

Logistic Regression      : Mean Accuracy = 0.5286 | Std = 0.0350
SVM (Linear)             : Mean Accuracy = 0.5762 | Std = 0.1245
SVM (RBF)                : Mean Accuracy = 0.5286 | Std = 0.0350
Random Forest            : Mean Accuracy = 0.5952 | Std = 0.1388
K-Nearest Neighbors      : Mean Accuracy = 0.5214 | Std = 0.1183
XGBoost                  : Mean Accuracy = 0.5905 | Std = 0.1683


In [75]:
from sklearn.model_selection import StratifiedKFold

print("Model Comparison using 10-Fold Cross Validation:\n")
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for name, model in models.items():
    accs, precs, recalls, f1s = [], [], [], []

    for train_index, test_index in skf.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)

        accs.append(accuracy_score(Y_test, Y_pred))
        precs.append(precision_score(Y_test, Y_pred))
        recalls.append(recall_score(Y_test, Y_pred))
        f1s.append(f1_score(Y_test, Y_pred))

    print(f"{name:25s}: "
          f"Accuracy = {np.mean(accs):.4f} ± {np.std(accs):.4f}, "
          f"Precision = {np.mean(precs):.4f}, "
          f"Recall = {np.mean(recalls):.4f}, "
          f"F1 Score = {np.mean(f1s):.4f}")

Model Comparison using 10-Fold Cross Validation:

Logistic Regression      : Accuracy = 0.5286 ± 0.0350, Precision = 0.5286, Recall = 1.0000, F1 Score = 0.6909
SVM (Linear)             : Accuracy = 0.6048 ± 0.1360, Precision = 0.6417, Recall = 0.7000, F1 Score = 0.6365
SVM (RBF)                : Accuracy = 0.5286 ± 0.0350, Precision = 0.5286, Recall = 1.0000, F1 Score = 0.6909
Random Forest            : Accuracy = 0.6667 ± 0.2043, Precision = 0.6733, Recall = 0.7583, F1 Score = 0.7067
K-Nearest Neighbors      : Accuracy = 0.5238 ± 0.1440, Precision = 0.5417, Recall = 0.7333, F1 Score = 0.6110
XGBoost                  : Accuracy = 0.6238 ± 0.1589, Precision = 0.6350, Recall = 0.7000, F1 Score = 0.6586


In [68]:
from sklearn.model_selection import GridSearchCV

params = {'C': [0.01, 0.1, 1, 10, 100]}
clf = GridSearchCV(LogisticRegression(max_iter=1000), params, cv=10)
clf.fit(X, Y)
print(clf.best_params_)


{'C': 0.1}


In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'degree': [2, 3, 4]  # Only used if kernel='poly'
}

# Initialize the model
svc = SVC()

# Set up GridSearchCV
grid = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit on data
grid.fit(X, Y)

# Best parameters and score
print("Best Parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)


Best Parameters: {'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
Best CV Accuracy: 0.6884615384615385
